In [2]:
import pandas as pd
import numpy as np

timeseries = pd.read_csv("insilico_size10_1_timeseries.tsv", sep='\t', header=0)

#the first column of timeseries is time, each experiment 21 rows
timeseries = np.array(timeseries)

In [6]:
timeseries.shape

(105, 10)

In [4]:
dif_time = np.unique(timeseries[:,0])[1] - np.unique(timeseries[:,0])[0]
M = np.unique(timeseries[:,0]).shape[0]
p = timeseries.shape[0]/M

In [5]:
timeseries = timeseries[:,1:]

In [66]:
#vec_y - vec of derivatives
def vec_y(timeseries = timeseries):
    x = timeseries.copy()
    #leave zeros rows
    y = np.zeros((M*p, n))
    for i in range(p):
        for j in range(1, M):
            y[i*M+j,:] = (x[i*M+j,:] - x[i*M+j-1,:])/dif_time
    return y    

In [8]:
p

5

### Preprocessing

## Algorithms

In [37]:
n = timeseries.shape[1]
def Phi_b(i, j, timeseries = timeseries):
    Fb_self = timeseries[i*M+j,:]
    Fb_connect = Fb_self/(1+Fb_self)
    Fb = np.hstack((Fb_self,Fb_connect))
    Fb = np.hstack((Fb, Fb_connect))
    I = np.eye(n)
    Phi_b = np.kron(Fb, I)
    return Phi_b
    

In [68]:
from scipy.linalg import block_diag
def matrix_Phi(timeseries = timeseries):
    phis = []
    for i in range(p):
        for j in range(M):
            phis.append(Phi_b(i,j))
    
    result = block_diag(*phis)
    return result

In [ ]:
# scalar shrinkage operator
# returns sgn(x) max(|x| - eps, 0)
def ScalarShrinkage(x, eps):
    return x[abs(x) < eps] = 0.0

# returns U ScalarShrinkage(sigma, eps) V^T
def S(x, eps):
    U, s, V = np.linalg.svd(x)
    return (U.dot(ScalarShrinkage(s, eps))).dot(V.T)

# orthogonal projector
def P(x, omega):
    r = np.zeros_like(x)
    for i, j in omega:
        r[i, j] = x[i, j]
    return r

In [ ]:
def compute_optproblem(B1, B2, E, W, l, alpha, omega,
                       N_ITERS = 10, nu1 = 3, nu2 = 3, p = 2.7, u0 = 1):
    
    def update_A(Ak, Wk, Ek, Y1k, Y2k, uk):
        m = Wk - Y1k / uk
        m = mpreshape(m, (300, 105))
        
        return S(1 / uk, Wk - Y1k / uk)
    
    def update_W(Ak, Wk, Ek, Y1k, Y2k, uk):
        PWk = P(B1.dot(W) + Ek - D + Y2k / uk, omega)
        x = Wk - ((B1.T.dot(PWk)) + Wk - Ak - Y1k / uk) / nu1
        return ScalarShrinkage(x, l / (uk * nu1))
    
    def update_E(Ak, Wk, Ek, Y1k, Y2k, uk):
        PEk = P(E + (B1.dot(Wk)) - D, omega)
        x = Ek - (PEk  + Y2k / uk) / nu2
        return ScalarShrinkage(x, alpha / (uk * nu2))
    
    def update_Y1(Ak, Wk, Ek, Y1k, Y2k, uk):        
        return Y1k + uk * P((B1.dot(Wk)) + Ek - D, omega)
    
    def update_Y2(Ak, Wk, Ek, Y1k, Y2k, uk):
        return Y2k + uk * (Ak - Wk)
    
    Ak = W.copy()
    Wk = 0
    Ek = 0
    Y1k = 0
    Y2k = 0
    uk = u0
    
    for i in range(N_ITERS):
        Ak = update_A(Ak, Wk, Ek, Y1k, Y2k, uk, l)
        Wk = update_W(Ak, Wk, Ek, Y1k, Y2k, uk, l)
        Ek = update_E(Ak, Wk, Ek, Y1k, Y2k, uk, l)
        Y1k = update_Y1(Ak, Wk, Ek, Y1k, Y2k, uk, l)
        Y2k = update_Y2(Ak, Wk, Ek, Y1k, Y2k, uk, l)
        uk *= p
        
    return Wk, Ek

In [ ]:
# set of indexes of non zero values
def supp(E):
    return set(map(tuple, np.column_stack(np.nonzero(E))))

def comute_all(B1, B2, E, W, l, alpha, omega, N_ITERS = 10):    
    Wk = W
    Ek = E
    
    # what is omega0
    omega_k = set()
    for i in range(N_ITERS):
        Wk, Ek = compute_optproblem(B1, B2, E, W, l, alpha, omega_k)
        omega_k = omega_k.difference(supp(Ek))
        
    return Wk

In [67]:
vec_y().shape

(1050, 1)

In [69]:
matrix_Phi().shape

(1050, 31500)